# Check_connection_of_UnrealCV

In [1]:
import time; print(time.strftime("The last update of this file: %Y-%m-%d %H:%M:%S", time.gmtime()))
import sys, time
# Establish connection with the UE4 game
from unrealcv import client
client.connect()
if not client.isconnected():
    print('UnrealCV server is not running. Run the game downloaded from http://unrealcv.github.io first.')
    sys.exit(-1)

# Checking status of connection between UnrealCV and UE4 game

res = client.request('vget /unrealcv/status')
# The image resolution and port is configured in the config file.
print('status: ',res)

The last update of this file: 2019-06-05 22:35:41


INFO:__init__:211:Got connection confirm: b'connected to test_game_4_19'


status:  Is Listening
Client Connected
9000
Configuration
Config file: C:/Program Files/Epic Games/UE_4.19/Engine/Binaries/Win64/unrealcv.ini
Port: 9000
Width: 640
Height: 480
FOV: 90.000000
EnableInput: true
EnableRightEye: false



In [2]:
import json
import numpy as np
import os
import math
import shutil
#import glob
import cv2
crop=0

# Hide the floor. It is doing for doing the crop of the actor flawlessly

In [3]:
hide_Floor=client.request('vset /object/Floor/hide')
hide_Floor_14=client.request('vset /object/Floor_14/hide')
print('hide_Floor: ',hide_Floor)
print('hide_Floor_14: ',hide_Floor_14)

hide_Floor:  ok
hide_Floor_14:  ok


# Function for cropping and making the primary text file for YOLO dataset

In [4]:
def do_crop(path_of_image,lit_image_name,mask_image_name,crop_image_type,class_number):
#    print("function_start!!!!!!!!!!!!")
    global crop
    split_lit_image_name=lit_image_name.split(".")
    cropped="cropped"
    
    lit_image=path_of_image+lit_image_name
    mask_image=path_of_image+mask_image_name
    read_lit_image=cv2.imread(lit_image)
    read_mask_image=cv2.imread(mask_image)
    
    image_mask_copy = read_mask_image.copy()
    imgray=cv2.cvtColor(read_mask_image,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(imgray,127,255,1)
#    print('Till now OK')
    image, contours, hierarchy =  cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image_mask_copy,contours,0,(0,255,0))
    
#    hsv = cv2.cvtColor(read_mask_image, cv2.COLOR_BGR2HSV)
#    hsv_channels = cv2.split(hsv)
#    _,thresh=cv2.threshold(hsv_channels[1],140,255,cv2.THRESH_BINARY_INV)
#    im2,contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#    cv2.drawContours(read_mask_image, contours, 0, (0,255,0), 3)
    if len(contours)>0:
        print('length is: ',len(contours))
        cnt = contours[0]
        x,y,w,h = cv2.boundingRect(cnt)
    
        draw_rec_lit_image=cv2.rectangle(read_lit_image,(x,y),(x+w,y+h),(255,255,255),1)
        crop_img = draw_rec_lit_image[y:y+h, x:x+w]
        crop=crop+1
        cv2.imwrite(str(path_of_image)+str(split_lit_image_name[0])+"_"+str(cropped)+"."+str(crop_image_type),crop_img)
        print('CROP done')
        
        f = open(path_of_image+str(split_lit_image_name[0])+'.txt', 'w')
        f.write('00'+str(class_number)+"\n")
        f.write(str(x)+' ')
        f.write(str(y)+' ')
        f.write(str(x+w)+' ')
        f.write(str(y+h)+' ')
        f.close()
        
        print('Pre Annotation File has made !!')
    else:
        print('here length is: ',len(contours))
        print('NO Crop has done')
        pass

# Loading information of actor from config file

In [5]:
with open('config_file_capture_image.json', 'r') as f:
    config = json.load(f)

polar_angle_start= config['DEFAULT']['polar_angle_start']
polar_angle_end = config['DEFAULT']['polar_angle_end']
polar_angle_step = config['DEFAULT']['polar_angle_step']
azimuthal_angle_start= config['DEFAULT']['azimuthal_angle_start']
azimuthal_angle_end= config['DEFAULT']['azimuthal_angle_end']
azimuthal_angle_step= config['DEFAULT']['azimuthal_angle_step']
#print('pola_step: ',polar_angle_step,'\t azi step: ',azimuthal_angle_step)
viewmode_1= config['DEFAULT']['viewmode_1']
viewmode_2= config['DEFAULT']['viewmode_2']
viewmode_3= config['DEFAULT']['viewmode_3']
#address= config['DEFAULT']['address']
image_type= config['DEFAULT']['image_type']

#actor_list= config['DEFAULT']['actor_list']
#actor_list=client.request(actor_list)
#print("type is:",type(address))
#print('actor: ',config['actor'])
actor_dict={}
for i in config['actor']:
    print(i)
    actor_dict[i]=[]
    actor_dict[i].append(polar_angle_start)
    actor_dict[i].append(polar_angle_end)
    actor_dict[i].append(azimuthal_angle_start)
    actor_dict[i].append(azimuthal_angle_end)
    actor_dict[i].append(config['actor'][i]['class'])
    actor_dict[i].append(config['actor'][i]['radius'])
print('actor dictionary: \n',actor_dict)


#print('type: ',type(actor_list),'\n list: ',actor_list)
#res_list=client.request('vget /objects')
#print('res_type: ',type(res_list),'res_list: ',res_list)

SM_CalgonitFinishKlarspueler_2
SM_CalgonitFinishMaschinenpfleger_5
SM_CalgonitFinish_2
SM_DenkMitGeschirrReiniger_2
SM_DenkMitGeschirrReinigerClassic_5
SM_DenkMitGeschirrReinigerNature_11
SM_DenkMitHygieneAllzweckreiniger_14
SM_DenkMitMaschinenpfleger_17
SM_DenkMitSpezialsalz_20
SM_SomatClassic_2
SM_DenkMitGeschirrReinigerEvo_8
SM_DomestosKraftUniversalreiniger_23
SM_HeitmannReineCitronenSaeure_26
SM_HeitmannReineCitronensaeure2_29
SM_SomatExtraAllIn1_32
SM_CalgonitFinishSpezialSalz_38
SM_CalgonitFinishVorratspack_41
SM_DenkMitEdelstahlReinigerSpray_44
SM_DenkMitEdelstahlreiniger_47
SM_DenkMitEntkalker_50
actor dictionary: 
 {'SM_CalgonitFinishKlarspueler_2': [90, 81, 0, 1, 1, 300], 'SM_CalgonitFinishMaschinenpfleger_5': [90, 81, 0, 1, 2, 300], 'SM_CalgonitFinish_2': [90, 81, 0, 1, 3, 300], 'SM_DenkMitGeschirrReiniger_2': [90, 81, 0, 1, 4, 300], 'SM_DenkMitGeschirrReinigerClassic_5': [90, 81, 0, 1, 5, 300], 'SM_DenkMitGeschirrReinigerNature_11': [90, 81, 0, 1, 6, 300], 'SM_DenkMitHygie

In [6]:
for i in actor_dict:
    print('Now: ',i,' will be hidden')
    hide=client.request('vset /object/'+str(i)+'/hide')
    print('hidden operation for ',i,' has done. \t',hide)

Now:  SM_CalgonitFinishKlarspueler_2  will be hidden
hidden operation for  SM_CalgonitFinishKlarspueler_2  has done. 	 ok
Now:  SM_CalgonitFinishMaschinenpfleger_5  will be hidden
hidden operation for  SM_CalgonitFinishMaschinenpfleger_5  has done. 	 ok
Now:  SM_CalgonitFinish_2  will be hidden
hidden operation for  SM_CalgonitFinish_2  has done. 	 ok
Now:  SM_DenkMitGeschirrReiniger_2  will be hidden
hidden operation for  SM_DenkMitGeschirrReiniger_2  has done. 	 ok
Now:  SM_DenkMitGeschirrReinigerClassic_5  will be hidden
hidden operation for  SM_DenkMitGeschirrReinigerClassic_5  has done. 	 ok
Now:  SM_DenkMitGeschirrReinigerNature_11  will be hidden
hidden operation for  SM_DenkMitGeschirrReinigerNature_11  has done. 	 ok
Now:  SM_DenkMitHygieneAllzweckreiniger_14  will be hidden
hidden operation for  SM_DenkMitHygieneAllzweckreiniger_14  has done. 	 ok
Now:  SM_DenkMitMaschinenpfleger_17  will be hidden
hidden operation for  SM_DenkMitMaschinenpfleger_17  has done. 	 ok
Now:  SM_D

In [7]:
for i in actor_dict:
    print('Now ',i,' will be appeared in the game')
    show_again=client.request('vset /object/'+str(i)+'/show')
    print('Visible actor is: ',i,'\t.',show_again)

Now  SM_CalgonitFinishKlarspueler_2  will be appeared in the game
Visible actor is:  SM_CalgonitFinishKlarspueler_2 	. ok
Now  SM_CalgonitFinishMaschinenpfleger_5  will be appeared in the game
Visible actor is:  SM_CalgonitFinishMaschinenpfleger_5 	. ok
Now  SM_CalgonitFinish_2  will be appeared in the game
Visible actor is:  SM_CalgonitFinish_2 	. ok
Now  SM_DenkMitGeschirrReiniger_2  will be appeared in the game
Visible actor is:  SM_DenkMitGeschirrReiniger_2 	. ok
Now  SM_DenkMitGeschirrReinigerClassic_5  will be appeared in the game
Visible actor is:  SM_DenkMitGeschirrReinigerClassic_5 	. ok
Now  SM_DenkMitGeschirrReinigerNature_11  will be appeared in the game
Visible actor is:  SM_DenkMitGeschirrReinigerNature_11 	. ok
Now  SM_DenkMitHygieneAllzweckreiniger_14  will be appeared in the game
Visible actor is:  SM_DenkMitHygieneAllzweckreiniger_14 	. ok
Now  SM_DenkMitMaschinenpfleger_17  will be appeared in the game
Visible actor is:  SM_DenkMitMaschinenpfleger_17 	. ok
Now  SM_De

In [8]:
show_Floor=client.request('vset /object/Floor/show')
show_Floor_14=client.request('vset /object/Floor_14/show')